In [41]:
import numpy as np

In [58]:
users_max_id = 6040
movie_max_id = 3952
correlations_data = np.fromfile('./correlations_data_set_saved', dtype=float)
correlations_data=correlations_data.reshape(movie_max_id+1,movie_max_id+1)

correlations_indicator = np.load('./correlations_indicator_set_saved.npy')
correlations_indicator=correlations_indicator.reshape(movie_max_id+1,movie_max_id+1)

bad_movies = np.load('./subsets/bad_movies.npy')
bad_movies = bad_movies.reshape((-1)).tolist()

subset_size=100
Indecies = np.array([x for x in range(0,movie_max_id+1)])

In [191]:
permutation= np.loadtxt('./subsets/randomsubset20')
permutation=permutation.reshape((-1)).astype(int)
bad_indices = np.zeros((permutation.shape[0]), dtype=bool)
for movie in bad_movies: 
    bad_indices = np.logical_or(permutation==movie,bad_indices)
permutation=permutation[~(bad_indices.reshape(-1))]
subset_size = permutation.shape[0]
ajdacency_matrix=np.zeros((subset_size+1,subset_size+1))
ajdacency_matrix=(correlations_indicator[permutation,:])[:,permutation]
print(permutation)
print(subset_size-1)

[   0 3811  918 3797 2740  344  537 3405 3068 1970  802 2694  927 1204
  900  733 1375 2417  960   35 2232 2837  751  807 3033  432 3054 3032
  338 1770 3134 2581 1853  188  779  964 1734 1824 1167  152 3936 2688
 3627 2990  357 3649 1449 2870 2284 1278 2362 2534 3755 1581 2925 3527
 1081  905 2278 1981 1342 3661  409  186 2751 1373  527 1074 2426 2674
 2000 3734 2839 1346 1715  280 1011  577  612 2465 3886 1814 2650 3181
 2103]
84


In [192]:
# ###############
# change W to be the head of the list we get
clustering_vector = np.zeros((20,movie_max_id+1))
for i in range(0,20):
    ajdacency_matrix_copy = np.copy(ajdacency_matrix)
    current_cluster_number = 0
    W = np.ones((subset_size))
    W[0]=0
    while np.sum(W) != 0:
        v = np.random.choice(np.argwhere(W>0).reshape(-1),size=1)[0]
        current_cluster_number = current_cluster_number + 1
        clustering_vector[ i,ajdacency_matrix_copy[0,v].astype(int)] = current_cluster_number
        movie_array = np.logical_and(ajdacency_matrix_copy[v,:]==1,W==1) 
        rand_array = np.random.rand(W.shape[0])
        movie_array = np.logical_and(movie_array==1,rand_array<=0.5)
        ajdacency_matrix_copy[1:,movie_array]=0
        ajdacency_matrix_copy[movie_array,1:]=0
        W[movie_array]=0
        clustering_vector[i,ajdacency_matrix_copy[0,movie_array].astype(int)]=current_cluster_number
        

In [193]:
costs=[]
for i in range(0,20):
    cost=0
    data_clusters = np.unique(clustering_vector[i,:])
    for c in range(1,len(data_clusters)+1):
        cluster = (clustering_vector[i,:] == c)
        if np.sum(cluster.astype(int))==1:
            cost=cost + np.log(1/(correlations_data[Indecies[cluster][0],Indecies[cluster][0]]))
        else:
            c_i=0
            for t in Indecies[cluster]:
                for j in Indecies[cluster]:
                    if t!=j:
                        if correlations_data[t,j]==0:
                                correlations_data[t,j]=correlations_data[j,t]
                        c_i=c_i+np.log(1/correlations_data[t,j])
            cost=cost+(c_i/2)*(1/(np.sum(cluster)-1))
    costs.append(cost)
np_costs = np.array(costs)
print('clustering {} is the best, with cost: {}'.format(np.argmin(np_costs),np.min(np_costs)))

clustering 2 is the best, with cost: 630.5330887070006
